### TensorFlow学习

网址：http://www.tensorfly.cn/tfdoc/get_started/basic_usage.html

### 1、综述

TensorFlow 是一个编程系统，使用图来表示计算任务。图中的节点被称之为op(operation 的缩写)。 一个op获得0个或多个Tensor，执行计算，产生0个或多个Tensor。每个Tensor是一个类型化的多维数组。例如，你可以将一小组图像集表示为一个四维浮点数数组，这四个维度分别是[batch, height, width, channels]。

一个TensorFlow图描述了计算的过程。为了进行计算，图必须在“会话”里被启动。“会话”将图的op分发到诸如CPU或GPU之类的设备上，同时提供执行op的方法。 这些方法执行后，将产生的tensor返回。在Python语言中，返回的tensor是numpy的ndarray对象；在C和C++语言中，返回的tensor是tensorflow::Tensor实例。

使用 TensorFlow, 必须明白TensorFlow:
* 计算模型——计算图： 使用图(graph）来表示计算任务。
* 运行模型——回话： 在被称之为会话(Session)的上下文(context)中执行图。
* 数据模型——张量： 使用tensor表示数据。
* 通过变量(Variable)维护状态。
* 使用feed和fetch可以为任意的操作(arbitrary operation)赋值或者从其中获取数据。

### 2、计算模型——计算图

构建图的第一步，是创建源op(source op)。 源op不需要任何输入，例如常量(Constant)。源op的输出被传递给其它op做运算。

Python库中,op构造器的返回值代表被构造出的op的输出,这些返回值可以传递给其它op构造器作为输入。

TensorFlow Python库有一个默认图(default graph),op构造器可以为其增加节点。这个默认图对许多程序来说已经足够用了。

In [1]:
import tensorflow as tf

# 创建一个常量op,产生一个1x2矩阵，这个op被作为一个节点
# 加到默认图中。
#
# 构造器的返回值代表该常量op的返回值。
matrix1 = tf.constant([[3., 3.]])

# 创建另外一个常量op,产生一个2x1矩阵。
matrix2 = tf.constant([[2.],[2.]])

# 创建一个矩阵乘法matmul op, 把'matrix1'和'matrix2'作为输入。
# 返回值'product'代表矩阵乘法的结果。
product = tf.matmul(matrix1, matrix2)

默认图现在有三个节点，两个constant() op，和一个matmul() op。为了真正进行矩阵相乘运算，并得到矩阵乘法的结果，必须在会话里启动这个图。

### 3、运行模型——会话

1）Session.run()启动图

构造阶段完成后，才能启动图。启动图的第一步是创建一个Session对象，如果无任何创建参数，会话构造器将启动默认图。

In [2]:
# 启动默认图.
sess = tf.Session()

# 调用sess的'run()'方法来执行矩阵乘法op,传入'product'作为该方法的参数。
# 上面提到,'product'代表了矩阵乘法op的输出,传入它是向方法表明,我们希望取回
# 矩阵乘法 op 的输出。
#
# 整个执行过程是自动化的,会话负责传递op所需的全部输入。op通常是并发执行的。
# 
# 函数调用'run(product)'触发了图中三个op(两个常量op和一个矩阵乘法op)的执行。
#
# 返回值'result'是一个numpy `ndarray`对象.
result = sess.run(product)
print(result)
# ==> [[ 12.]]

# 任务完成, 关闭会话.
sess.close()

[[ 12.]]


在实现上, TensorFlow 将图形定义转换成分布式执行的操作, 以充分利用可用的计算资源(如CPU或GPU)。 一般你不需要显式指定使用CPU还是GPU，TensorFlow能自动检测。如果检测到GPU,TensorFlow会尽可能地利用找到的第一个GPU来执行操作。

如果机器上有超过一个可用的GPU,除第一个外的其它GPU默认是不参与计算的。为了让TensorFlow使用这些GPU,你必须将op明确指派给它们执行。with...Device语句用来指派特定的CPU或GPU执行操作：

In [3]:
# with tf.Session() as sess:
#   with tf.device("/gpu:1"):
#     matrix1 = tf.constant([[3., 3.]])
#     matrix2 = tf.constant([[2.],[2.]])
#     product = tf.matmul(matrix1, matrix2)
#     ...

设备用字符串进行标识. 目前支持的设备包括:
* "/cpu:0": 机器的CPU
* "/gpu:0": 机器的第一个GPU,如果有的话
* "/gpu:1": 机器的第二个GPU,以此类推


2）交互式启动图

文档中的Python示例使用一个会话Session来启动图,并调用Session.run()方法执行操作。

为了便于使用诸如IPython之类的Python交互环境,可以使用InteractiveSession代替Session类,使用Tensor.eval()和Operation.run()方法代替Session.run()，这样可以避免使用一个变量来持有会话。

In [4]:
# 进入一个交互式TensorFlow会话.
import tensorflow as tf
sess = tf.InteractiveSession()

x = tf.Variable([1.0, 2.0])
a = tf.constant([3.0, 3.0])

# 使用初始化器initializer op的run()方法初始化'x' 
x.initializer.run()

# 增加一个减法subtract op,从'x'减去'a'，运行减法op,输出结果 
sub = tf.subtract(x, a)
print(sub.eval())
# ==> [-2. -1.]

[-2. -1.]


### 4、数据模型——张量

TensorFlow程序使用tensor数据结构来代表所有的数据,计算图中,操作间传递的数据都是tensor，你可以把,TensorFlow的数据模型tensor看作是一个n维的数组或列表。 一个tensor包含一个静态类型rank,和一个shape。